## 1. Import packages

In [1]:
#!pip install tweepy

In [2]:
import os
import tweepy as tw
import pandas as pd
from tqdm import tqdm

## 2. Twitter API authentication

In [3]:
consumer_api_key = os.environ["TWITTER_CONSUMER_API_KEY"]
consumer_api_secret = os.environ["TWITTER_CONSUMER_API_SECRET"]

In [4]:
auth = tw.OAuthHandler(consumer_api_key, consumer_api_secret)

In [5]:
api = tw.API(auth, wait_on_rate_limit=True)

## 3. Tweets query

### 3.1. Define the query

In [6]:
search_words = "#covid19 -filter:retweets"
date_since = "2020-03-01"
# Collect tweets
tweets = tw.Cursor(api.search,
              q=search_words,
              lang="en",
              since=date_since).items(7500)

### 3.2. Retreive the tweets

In [7]:
tweets_copy = []
for tweet in tqdm(tweets):
    tweets_copy.append(tweet)

7500it [15:37,  8.00it/s]


In [8]:
print(f"new tweets retrieved: {len(tweets_copy)}")

new tweets retrieved: 7500


## 4. Populate the dataset

In [9]:
tweets_df = pd.DataFrame()
for tweet in tqdm(tweets_copy):
    hashtags = []
    try:
        for hashtag in tweet.entities["hashtags"]:
            hashtags.append(hashtag["text"])
    except:
        pass
    tweets_df = tweets_df.append(pd.DataFrame({'user_name': tweet.user.name, 
                                               'user_location': tweet.user.location,\
                                               'user_description': tweet.user.description,
                                               'user_created': tweet.user.created_at,
                                               'user_followers': tweet.user.followers_count,
                                               'user_friends': tweet.user.friends_count,
                                               'user_favourites': tweet.user.favourites_count,
                                               'user_verified': tweet.user.verified,
                                               'date': tweet.created_at,
                                               'text': tweet.text, 
                                               'hashtags': [hashtags if hashtags else None],
                                               'source': tweet.source,
                                               'is_retweet': tweet.retweeted}, index=[0]))

100%|██████████| 7500/7500 [00:37<00:00, 198.99it/s]


In [10]:
tweets_df.head()

user_name       user_location  \
0                           Marley                       
0  Revolution Is Human Evolution 🌹                       
0                        HarryQuim                       
0               Suzanne Winchester  Salt Lake City, UT   
0                  70 Million Jobs     Los Angeles, CA   

                                    user_description        user_created  \
0                                                    2020-01-27 01:17:53   
0  a.k.a. CuriosityKilledtheCat@9Times Later & Ma... 2020-06-11 04:12:23   
0           Retweets not necessarily an endorsement. 2017-06-15 12:33:30   
0  Media Relations focusing on the world of healt... 2009-07-14 19:43:18   
0  First national, for-profit employment platform... 2016-06-15 15:43:48   

   user_followers  user_friends  user_favourites  user_verified  \
0              56           587            50037          False   
0             309           822             4411          False   
0             490           546             3873          False   
0             519           921             6196          False   
0            1281           798             2432          False   

                 date                                               text  \
0 2020-08-01 19:21:48  @SenCoryGardner While Colorado turns 144 today...   
0 2020-08-01 19:21:47  Obscene is the word for it. So are corrupt, av...   
0 2020-08-01 19:21:47  Meet another knob head. Labour MP @JonAshworth...   
0 2020-08-01 19:21:45  In triple-digit temperatures, hospitals ramp u...   
0 2020-08-01 19:21:41  As of July 28, nearly 79,000 people in prison ...   

    hashtags               source  is_retweet  
0  [COVID19]   Twitter for iPhone       False  
0       None  Twitter for Android       False  
0       None      Twitter Web App       False  
0       None   Twitter for iPhone       False  
0  [COVID19]            TweetDeck       False

## 5. Save the data

### 5.1. Read past data

In [11]:
tweets_old_df = pd.read_csv("covid19_tweets.csv")
print(f"past tweets: {tweets_old_df.shape}")

past tweets: (51936, 13)


### 5.2. Merge past and present data

In [12]:
tweets_all_df = pd.concat([tweets_old_df, tweets_df], axis=0)
print(f"new tweets: {tweets_df.shape[0]} past tweets: {tweets_old_df.shape[0]} all tweets: {tweets_all_df.shape[0]}")

new tweets: 7500 past tweets: 51936 all tweets: 59436


### 5.3. Drop duplicates

In [13]:
tweets_all_df.drop_duplicates(subset = ["user_name", "date", "text"], inplace=True)
print(f"all tweets: {tweets_all_df.shape}")

all tweets: (59436, 13)


### 5.4. Export the updated data

In [14]:
tweets_all_df.to_csv("covid19_tweets.csv", index=False)